In [ ]:
# NOTE: use this to remove the watermark and logo then merge all the image together

import os
import requests
from io import BytesIO
import cv2
import numpy as np


def requestImageV2(
    picHeight,
    picWidth,
    zoom,
    scale,
    maptype,
    lat,
    lng,
    row,
    col,
    image_prefix,
    picHeightPadded=640,
):

    center = str(lat) + "," + str(lng)
    url = (
        "https://maps.googleapis.com/maps/api/staticmap?center="
        + center
        + "&zoom="
        + str(zoom)
        + "&size="
        + str(picWidth)
        + "x"
        + str(picHeightPadded)
        + "&key="
        + os.getenv("api_key")
        + "&maptype="
        + maptype
        + "&scale="
        + str(scale)
    )
    dir_path = os.getcwd()
    filename = f"{dir_path}/output/" + image_prefix + str(col) + "," + str(row) + ".png"
    folder = os.path.dirname(filename)
    # load r.content as image then crop the image to picHeight
    r = requests.get(url)

    assert r.status_code == 200, "expected status code 200, but got " + str(
        r.status_code
    )
    os.makedirs(folder, exist_ok=True)
    image = Image.open(BytesIO(r.content))
    # rgb_image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
    # image = Image.fromarray(rgb_image)
    image = image.crop(
        (
            0,
            (picHeightPadded - picHeight) // 2 * scale,
            picWidth * scale,
            (picHeightPadded - (picHeightPadded - picHeight) // 2) * scale,
        )
    )
    image.save(filename)

    print("writtern to file: " + filename)

In [ ]:
import requests
import math
from dotenv import load_dotenv
import os

import math


def create_boundary_latlng(lat, lng, r):
    # Radius of the Earth in meters
    R = 6371000

    # Convert lat and lng from degrees to radians
    lat_rad = math.radians(lat)
    lng_rad = math.radians(lng)

    # Latitude offset in radians
    delta_lat = r / R

    # Longitude offset in radians (adjusted by latitude)
    delta_lng = r / (R * math.cos(lat_rad))

    # Convert offsets back to degrees
    delta_lat_deg = math.degrees(delta_lat)
    delta_lng_deg = math.degrees(delta_lng)

    # Calculate boundary coordinates
    min_lat = lat - delta_lat_deg
    max_lat = lat + delta_lat_deg
    min_lng = lng - delta_lng_deg
    max_lng = lng + delta_lng_deg

    return min_lat, min_lng, max_lat, max_lng


def latLngToPoint(mapWidth, mapHeight, lat, lng):

    x = (lng + 180) * (mapWidth / 360)
    y = (
        (
            1
            - math.log(
                math.tan(lat * math.pi / 180) + 1 / math.cos(lat * math.pi / 180)
            )
            / math.pi
        )
        / 2
    ) * mapHeight

    return (x, y)


def pointToLatLng(mapWidth, mapHeight, x, y):

    lng = x / mapWidth * 360 - 180

    n = math.pi - 2 * math.pi * y / mapHeight
    lat = 180 / math.pi * math.atan(0.5 * (math.exp(n) - math.exp(-n)))

    return (lat, lng)


def getImageBounds(mapWidth, mapHeight, xScale, yScale, lat, lng):

    centreX, centreY = latLngToPoint(mapWidth, mapHeight, lat, lng)

    southWestX = centreX - (mapWidth / 2) / xScale
    southWestY = centreY + (mapHeight / 2) / yScale
    SWlat, SWlng = pointToLatLng(mapWidth, mapHeight, southWestX, southWestY)

    northEastX = centreX + (mapWidth / 2) / xScale
    northEastY = centreY - (mapHeight / 2) / yScale
    NElat, NElng = pointToLatLng(mapWidth, mapHeight, northEastX, northEastY)

    return [SWlat, SWlng, NElat, NElng]


def getLatStep(mapWidth, mapHeight, yScale, lat, lng):

    pointX, pointY = latLngToPoint(mapWidth, mapHeight, lat, lng)

    steppedPointY = pointY - ((mapHeight) / yScale)
    newLat, originalLng = pointToLatLng(mapWidth, mapHeight, pointX, steppedPointY)

    latStep = lat - newLat

    return latStep


def requestImage(
    picHeight,
    picWidth,
    zoom,
    scale,
    maptype,
    lat,
    lng,
    row,
    col,
    image_prefix="image_prefix",
):

    center = str(lat) + "," + str(lng)
    url = (
        "https://maps.googleapis.com/maps/api/staticmap?center="
        + center
        + "&zoom="
        + str(zoom)
        + "&size="
        + str(picWidth)
        + "x"
        + str(picHeight)
        + "&key="
        + os.getenv("api_key")
        + "&maptype="
        + maptype
        + "&scale="
        + str(scale)
    )
    dir_path = os.getcwd()
    filename = f"{dir_path}/output/" + image_prefix + str(col) + "," + str(row) + ".png"
    folder = os.path.dirname(filename)
    os.makedirs(folder, exist_ok=True)
    r = requests.get(url)
    f = open(filename, "wb")
    f.write(r.content)
    f.close()

    print("writtern to file: " + filename)


load_dotenv()
# Bounding box for area to be scanned. image_prefix is added to file name.
image_prefix = "SanFran"
# center: { lat: 13.91365211, lng: 99.67114584 },
# padding = 0.003

# # northWestLat = 37.806716
# # northWestLng = -122.477702
# # southEastLat = 37.7636132
# # southEastLng = -122.4319237
# # 15.37666892	100.2520627
# lat = 15.37666892
# lon = 100.2520627
# northWestLat = lat + padding
# northWestLng = lon - padding
# southEastLat = lat - padding
# southEastLng = lon + padding
distance_m = 150
central_lat = 15.37666892
central_lng = 100.2520627
southEastLat, northWestLng, northWestLat, southEastLng = create_boundary_latlng(
    central_lat, central_lng, distance_m
)


assert os.getenv("api_key") is not None, "Please set your API key in .env file"
# Variables for API request (more info in README)

zoom = 19
picHeight = 550
picWidth = 550
scale = 2
maptype = "roadmap"

# --- do not change variables below this point ---

mapHeight = 256
mapWidth = 256
print("START")


def create_tile_images(
    zoom,
    picHeight,
    picWidth,
    scale,
    maptype,
    northWestLat,
    northWestLng,
    southEastLat,
    southEastLng,
    image_prefix,
):
    xScale = math.pow(2, zoom) / (picWidth / mapWidth)
    yScale = math.pow(2, zoom) / (picHeight / mapWidth)

    startLat = northWestLat
    startLng = northWestLng

    startCorners = getImageBounds(
        mapWidth, mapHeight, xScale, yScale, startLat, startLng
    )
    lngStep = startCorners[3] - startCorners[1]

    col = 0
    lat = startLat

    while lat >= southEastLat:
        lng = startLng
        row = 0

        while lng <= southEastLng:
            requestImageV2(
                picHeight,
                picWidth,
                zoom,
                scale,
                maptype,
                lat,
                lng,
                row,
                col,
                image_prefix,
            )

            row = row + 1
            lng = lng + lngStep

        col = col + 1
        lat = lat + getLatStep(mapWidth, mapHeight, yScale, lat, lng)

In [ ]:
# Example usage
central_lat = 15.37666892  # Latitude of the center point
central_lng = 100.2520627  # Longitude of the center point
distance_m = 150  # Distance in meters

min_lat, min_lng, max_lat, max_lng = create_boundary_latlng(
    central_lat, central_lng, distance_m
)
print(f"Min Latitude: {min_lat}, Min Longitude: {min_lng}")
print(f"Max Latitude: {max_lat}, Max Longitude: {max_lng}")
print(max_lat - min_lat)
print(max_lng - min_lng)

In [ ]:
create_tile_images(
    zoom,
    picHeight,
    picWidth,
    scale,
    maptype,
    northWestLat,
    northWestLng,
    southEastLat,
    southEastLng,
    image_prefix,
)
print("COMPLETE")

In [ ]:
def create_tile_images_from_middle(
    zoom, picHeight, picWidth, scale, maptype, lat, lon, radius, image_prefix
):
    mapHeight = 256
    mapWidth = 256

    # Calculate the bounds of the area to be covered
    southEastLat, northWestLng, northWestLat, southEastLng = create_boundary_latlng(
        lat, lon, radius
    )
    print(southEastLat, northWestLng, northWestLat, southEastLng)
    # Calculate the size of a single tile in degrees
    xScale = math.pow(2, zoom) / (picWidth / mapWidth)
    yScale = math.pow(2, zoom) / (picHeight / mapHeight)
    tile_bounds = getImageBounds(mapWidth, mapHeight, xScale, yScale, lat, lon)
    tile_width = tile_bounds[3] - tile_bounds[1]
    tile_height = tile_bounds[2] - tile_bounds[0]
    print(tile_bounds)
    print(tile_width, tile_height)

    # Calculate the number of tiles needed in each direction
    tiles_x = math.ceil((southEastLng - northWestLng) / tile_width)
    tiles_y = math.ceil((northWestLat - southEastLat) / tile_height)

    # Ensure we have an odd number of tiles if necessary
    tiles_x = tiles_x + 1 if tiles_x % 2 == 0 else tiles_x
    tiles_y = tiles_y + 1 if tiles_y % 2 == 0 else tiles_y

    print(tiles_x)
    print(tiles_y)
    # Calculate the starting coordinates for the top-left tile
    start_lat = lat + (tile_height * (tiles_y - 1) / 2)
    start_lng = lon - (tile_width * (tiles_x - 1) / 2)
    for col in range(tiles_y):
        for row in range(tiles_x):
            current_lat = start_lat - (col * tile_height)
            current_lng = start_lng + (row * tile_width)

            requestImageV2(
                picHeight,
                picWidth,
                zoom,
                scale,
                maptype,
                current_lat,
                current_lng,
                row,
                col,
                image_prefix,
            )

    print(
        f"Created {tiles_x * tiles_y} tiles covering a radius of {radius} meters around ({lat}, {lon})"
    )


distance_m = 500
create_tile_images_from_middle(
    zoom,
    picHeight,
    picWidth,
    scale,
    maptype,
    central_lat,
    central_lng,
    distance_m,
    image_prefix,
)

In [ ]:
import math


import math


def calculate_tile_size_meters_thailand(
    zoom: int,
    tile_size: int,
) -> float:
    # Earth's circumference in meters
    earth_circumference = 40075017

    # Fixed latitude for Thailand
    thailand_standard_lat = 15.8700  # 15.8700 degrees North

    # Ground resolution at the equator
    ground_resolution = earth_circumference / (256 * 2**zoom)

    # Adjust for Thailand's standard latitude
    adjusted_resolution = ground_resolution * math.cos(
        math.radians(thailand_standard_lat)
    )

    # Calculate dimensions in meters
    image_side_in_meter = tile_size * adjusted_resolution

    return image_side_in_meter


# Example usage:
zoom = 19  # From your original code
picWidth = 550  # From your original code
picHeight = 550  # From your original code

width_meters = calculate_tile_size_meters_thailand(zoom, picWidth)
print(
    f"Approximate tile size for Thailand: {width_meters:.2f} x {width_meters:.2f} meters"
)

In [ ]:
from PIL import Image


def crop_square_image_to_meter(
    target_meter: int,
    image_side_in_meter: int,
    image: Image.Image,
) -> Image.Image:

    # Ensure target size is reasonable
    if target_meter > image_side_in_meter:
        raise ValueError(
            "Target meter size cannot be larger than the image's smaller dimension."
        )

    # Calculate pixel crop size based on target meter size and image ratio
    target_pixels = int(target_meter * image.width / image_side_in_meter)

    # Handle potential aspect ratio mismatch
    if target_pixels > image.height:
        target_pixels = image.height

    # Calculate starting coordinates for centered crop
    left = (image.width - target_pixels) // 2
    top = (image.height - target_pixels) // 2

    # Perform the crop
    crop_image = image.crop((left, top, left + target_pixels, top + target_pixels))

    return crop_image

In [ ]:
# # requestImageV2(picHeight, picWidth, zoom, scale, maptype, lat, lng, row, col,"test")
# picHeightPadded=640
# center = str(lat) + "," + str(lng)
# url = (
#     "https://maps.googleapis.com/maps/api/staticmap?center="
#     + center
#     + "&zoom="
#     + str(zoom)
#     + "&size="
#     + str(picWidth)
#     + "x"
#     + str(picHeightPadded)
#     + "&key="
#     + os.getenv("api_key")
#     + "&maptype="
#     + maptype
#     + "&scale="
#     + str(scale)
# )
# dir_path = os.getcwd()
# filename = f"{dir_path}/output/" + image_prefix + str(col) + "," + str(row) + ".png"
# folder = os.path.dirname(filename)
# # load r.content as image then crop the image to picHeight
# r = requests.get(url)

# assert r.status_code == 200, "expected status code 200, but got " + str(r.status_code)
# os.makedirs(folder, exist_ok=True)
# image = Image.open(BytesIO(r.content))
# rgb_image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
# image = Image.fromarray(rgb_image)
# image = image.crop(
#     (
#         0,
#         (picHeightPadded - picHeight) // 2*scale,
#         picWidth*scale,
#         (picHeightPadded - (picHeightPadded - picHeight) // 2)*scale,
#     )
# )
# image.save(filename)

# print("writtern to file: " + filename)

In [ ]:
# image = Image.open(BytesIO(r.content))
# # rgb_image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
# # image = Image.fromarray(rgb_image)
# image = image.crop(
#     (
#         0,
#         (picHeightPadded - picHeight) // 2*scale,
#         picWidth*scale,
#         (picHeightPadded - (picHeightPadded - picHeight) // 2)*scale,
#     )
# )
# image.save(filename)

In [ ]:
image_folder_path = (
    "/Users/ford/Documents/coding/cro_location_intelligence/save_google_map/output"
)

image_filename_list = os.listdir(image_folder_path)
image_filename_list = [i for i in image_filename_list if i.endswith(".png")]

In [ ]:
len(image_filename_list)

In [ ]:
sqrt_len = math.ceil(math.sqrt(len(image_filename_list)))

In [ ]:
image_filename_list

In [ ]:
image_filename_list.sort()
import cv2
import numpy as np
from PIL import Image


# create function to merge grid of images
def merge_grid_image(image_filename_list, grid_size) -> Image.Image:
    assert grid_size**2 == len(
        image_filename_list
    ), "grid_size must be square root of number of images"
    image_list = []
    for i in range(grid_size):
        row_image_list = []
        for j in range(grid_size):
            image_path = (
                image_folder_path + "/" + image_filename_list[i * grid_size + j]
            )
            # read image as bgr
            img = cv2.imread(
                image_path,
            )
            # convert to rgb
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            row_image_list.append(img)
        image_list.append(np.concatenate(row_image_list, axis=1))
    full_image = np.concatenate(image_list, axis=0)
    full_image = Image.fromarray(full_image)
    return full_image


full_image = merge_grid_image(image_filename_list, sqrt_len)

In [ ]:
# save merge image
full_image.save("full_image_2.png")

In [ ]:
# create function to display image
import matplotlib.pyplot as plt


def display_image(image: Image.Image, plot_size=(10, 10)):
    plt.figure(figsize=plot_size)
    plt.imshow(image)
    plt.axis("off")
    plt.show()

In [ ]:
display_image(full_image)

In [ ]:
h, w = full_image.size
h, w
image_side = min(h, w)
image_side

In [ ]:
image_side_in_meter = calculate_tile_size_meters_thailand(zoom, image_side)
print(image_side_in_meter)
crop_image = crop_square_image_to_meter(1000, image_side_in_meter, full_image)

In [ ]:
crop_image.size

In [ ]:
crop_image